In [6]:
import fmpsdk as fmp
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv("./scripts/.env")
fmp_key = os.environ.get("FMP_KEY")

In [1]:
from scripts import treasuries

In [2]:
treasuries.yields_data()

,1 MO,2 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,,
1990-01-02,NaN,NaN,7.83,7.89,7.81,7.87,7.90,7.87,7.98,7.94,NaN,8.00
1990-01-03,NaN,NaN,7.89,7.94,7.85,7.94,7.96,7.92,8.04,7.99,NaN,8.04
1990-01-04,NaN,NaN,7.84,7.90,7.82,7.92,7.93,7.91,8.02,7.98,NaN,8.04
1990-01-05,NaN,NaN,7.79,7.85,7.79,7.90,7.94,7.92,8.03,7.99,NaN,8.06
1990-01-08,NaN,NaN,7.79,7.88,7.81,7.90,7.95,7.92,8.05,8.02,NaN,8.09
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-29,5.60,5.59,5.40,5.26,4.79,4.23,4.01,3.84,3.88,3.88,4.20,4.03
2024-01-02,5.55,5.54,5.46,5.24,4.80,4.33,4.09,3.93,3.95,3.95,4.25,4.08
2024-01-03,5.54,5.54,5.48,5.25,4.81,4.33,4.07,3.90,3.92,3.91,4.21,4.05


In [7]:
symbol="AMD"

In [8]:
company_profile = fmp.company_profile(apikey=fmp_key, symbol=symbol)
last = company_profile[0]['price']
mcap = company_profile[0]['mktCap']
industry = company_profile[0]['industry']

In [9]:
last, mcap, industry

(167.14, 270063140600, 'Semiconductors')

In [10]:
# Retrieve key metrics
key_metrics = fmp.key_metrics_ttm(apikey=fmp_key, symbol=symbol)
eps_ttm = round(key_metrics[0]['netIncomePerShareTTM'], 2)
pe_ttm = round(key_metrics[0]['peRatioTTM'], 2)

In [11]:
eps_ttm, pe_ttm

(0.52, 318.82)

In [12]:
duration = "quarter"

In [13]:
# Retrieve income statement
inc_statement = fmp.income_statement(apikey=fmp_key, symbol=symbol, period=duration)
shares = inc_statement[0]['weightedAverageShsOut']
net_income = inc_statement[0]['netIncome']

In [14]:
# Retrieve balance sheet
financials = fmp.balance_sheet_statement(apikey=fmp_key, symbol=symbol, period=duration)
current_assets = financials[0]['cashAndShortTermInvestments'] + financials[0]['inventory'] + \
                    (financials[0]['netReceivables'] / 2) + financials[0]['otherCurrentAssets']
cash_share = round(financials[0]['cashAndCashEquivalents'] / shares, 2)
total_ratio = round(financials[0]['totalAssets'] / financials[0]['totalLiabilities'], 2)
net_current_assets = current_assets - financials[0]['totalCurrentLiabilities']
liability_share = round(financials[0]['totalCurrentLiabilities'] / shares, 2)
tangible_assets = (financials[0]['propertyPlantEquipmentNet'] / 2) + net_current_assets
inv_yield = round(net_income / mcap, 3)
operating_margin = inc_statement[0]['operatingIncomeRatio']
bvps = round(tangible_assets / shares, 2)
bvps_price = bvps / last
current_ratio = round(financials[0]['totalCurrentAssets'] / financials[0]['totalCurrentLiabilities'], 2)

In [15]:
# Retrieve earnings surprise
earnings = fmp.earnings_surprises(apikey=fmp_key, symbol=symbol)
earning = pd.DataFrame.from_dict(earnings)
earning.index = earning['date']
earning = earning.drop(columns='date')
earning = earning.sort_index()
fq_eps = earning.iloc[-5:, 1].mean()

In [16]:
# Create data frames
company_health = {'BVPS': [bvps], 'Cash/share': [cash_share], 'liability_share': [liability_share],
                    'EPS_ttm': [eps_ttm], 'fq_eps': [fq_eps], 'last': [last], 'industry': [industry]}
company_valuation = {'BVPS/Price': [bvps_price], 'P/E': [pe_ttm], 'Inv_Yield': [inv_yield], 'last': [last],
                        'Current_ratio': [current_ratio], 'Total_Ratio': [total_ratio], 'industry': [industry]}

health_df = pd.DataFrame(company_health, index=[symbol])
valuation_df = pd.DataFrame(company_valuation, index=[symbol])

In [17]:
health_df

,BVPS,Cash/share,liability_share,EPS_ttm,fq_eps,last,industry
AMD,5.26,2.43,4.14,0.52,0.684,167.14,Semiconductors


In [18]:
type(valuation_df)

pandas.core.frame.DataFrame

In [ ]:
import fmpsdk as fmp
import pandas as pd

def get_company_data(symbol, duration):
    fmp_key = 'd7772eac71bcd3ae5dac1dc4e58e0a5c'
    
    # Retrieve company profile
    company_profile = fmp.company_profile(apikey=fmp_key, symbol=symbol)
    last = company_profile[0]['price']
    mcap = company_profile[0]['mktCap']
    industry = company_profile[0]['industry']
    
    # Retrieve key metrics
    key_metrics = fmp.key_metrics_ttm(apikey=fmp_key, symbol=symbol)
    eps_ttm = round(key_metrics[0]['netIncomePerShareTTM'], 2)
    pe_ttm = round(key_metrics[0]['peRatioTTM'], 2)
    
    # Retrieve income statement
    inc_statement = fmp.income_statement(apikey=fmp_key, symbol=symbol, period=duration)
    shares = inc_statement[0]['weightedAverageShsOut']
    net_income = inc_statement[0]['netIncome']
    
    # Retrieve balance sheet
    financials = fmp.balance_sheet_statement(apikey=fmp_key, symbol=symbol, period=duration)
    current_assets = financials[0]['cashAndShortTermInvestments'] + financials[0]['inventory'] + \
                     (financials[0]['netReceivables'] / 2) + financials[0]['otherCurrentAssets']
    cash_share = round(financials[0]['cashAndCashEquivalents'] / shares, 2)
    total_ratio = round(financials[0]['totalAssets'] / financials[0]['totalLiabilities'], 2)
    net_current_assets = current_assets - financials[0]['totalCurrentLiabilities']
    liability_share = round(financials[0]['totalCurrentLiabilities'] / shares, 2)
    tangible_assets = (financials[0]['propertyPlantEquipmentNet'] / 2) + net_current_assets
    inv_yield = round(net_income / mcap, 3)
    operating_margin = inc_statement[0]['operatingIncomeRatio']
    bvps = round(tangible_assets / shares, 2)
    bvps_price = bvps / last
    current_ratio = round(financials[0]['totalCurrentAssets'] / financials[0]['totalCurrentLiabilities'], 2)
    
    # Retrieve earnings surprise
    earnings = fmp.earnings_surprises(apikey=fmp_key, symbol=symbol)
    earning = pd.DataFrame.from_dict(earnings)
    earning.index = earning['date']
    earning = earning.drop(columns='date')
    earning = earning.sort_index()
    fq_eps = earning.iloc[-5:, 1].mean()
    
    # Create data frames
    company_health = {'BVPS': [bvps], 'Cash/share': [cash_share], 'liability_share': [liability_share],
                      'EPS_ttm': [eps_ttm], 'fq_eps': [fq_eps], 'last': [last], 'industry': [industry]}
    company_valuation = {'BVPS/Price': [bvps_price], 'P/E': [pe_ttm], 'Inv_Yield': [inv_yield], 'last': [last],
                         'Current_ratio': [current_ratio], 'Total_Ratio': [total_ratio], 'industry': [industry]}
    
    health_df = pd.DataFrame(company_health, index=[symbol])
    valuation_df = pd.DataFrame(company_valuation, index=[symbol])
    
    return health_df, valuation_df

def concat_stocks(data_health, data_valuation, symbol, duration):
    stock_health, stock_valuation = get_company_data(symbol, duration)
    health_frame = pd.concat([data_health, stock_health])
    valuation_frame = pd.concat([data_valuation, stock_valuation])
    
    return health_frame, valuation_frame

# Example usage:
symbol = 'AAPL'  # Example stock symbol
duration = 'annual'  # Example duration

# Initial data frames
initial_health_df = pd.DataFrame()
initial_valuation_df = pd.DataFrame()

# Concatenating stocks
final_health_df, final_valuation_df = concat_stocks(initial_health_df, initial_valuation_df, symbol, duration)